In [2]:
import pandas as pd

In [ ]:
def threshold_ema_strategy(full_dataset_file, session_length=7, threshold_pct=0.5, min_slope=0.1):

    data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    
    data['daily_return'] = data['Cumulative_Return'].diff()
    data['ema5_slope'] = data['EMA_5'].pct_change() * 100  
    data['ema20_slope'] = data['EMA_20'].pct_change() * 100
    
    data['signal'] = 0
    data['position'] = 0
    data['consecutive_sells'] = 0
    
    for i in range(2, len(data)):
        ema5 = data.at[i, 'EMA_5']
        ema20 = data.at[i, 'EMA_20']
        ema50 = data.at[i, 'EMA_50']
        slope5 = data.at[i, 'ema5_slope']
        slope20 = data.at[i, 'ema20_slope']
        prev_ema5 = data.at[i-1, 'EMA_5']
        prev_ema20 = data.at[i-1, 'EMA_20']
        
        ema5_above_20 = (ema5 > ema20 * (1 + threshold_pct/100))
        ema20_above_50 = (ema20 > ema50 * (1 + threshold_pct/100))
        strong_slope = (slope5 > min_slope) and (slope20 > min_slope/2)
        crossing_up = (ema5 > ema20) and (prev_ema5 <= prev_ema20)
        
        buy_signal = sum([
            ema5_above_20,
            ema20_above_50, 
            strong_slope,
            crossing_up
        ]) >= 2
        
        ema5_below_20 = (ema5 < ema20 * (1 - threshold_pct/100))
        negative_slope = (slope5 < -min_slope)
        
        if buy_signal and not (ema5_below_20 or negative_slope):
            data.at[i, 'signal'] = 1
            data.at[i, 'consecutive_sells'] = 0
        else:
            data.at[i, 'signal'] = 0
            data.at[i, 'consecutive_sells'] = data.at[i-1, 'consecutive_sells'] + 1
    
    in_session = False
    for i in range(1, len(data)):
        if not in_session:
            if data.at[i, 'signal'] == 1:
                in_session = True
                data.at[i, 'position'] = 1
        else:
            if (data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - data[data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                data.at[i, 'position'] = 0
            else:
                data.at[i, 'position'] = 1
    
    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nTHRESHOLD-BASED EMA STRATEGY")
    print(f"Parameters: Threshold={threshold_pct}%, Min Slope={min_slope}%")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data